In [18]:
import requests
import pg8000
import ssl

In [19]:

r = requests.get('http://api.geonames.org/search?featureClass=P&country=FR&maxRows=10&orderby=population&type=json&username=yoannbouge'); 

result = r.json()['geonames']
result

[{'adminCode1': '11',
  'lng': '2.3488',
  'geonameId': 2988507,
  'toponymName': 'Paris',
  'countryId': '3017382',
  'fcl': 'P',
  'population': 2138551,
  'countryCode': 'FR',
  'name': 'Paris',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': 'IDF'},
  'countryName': 'France',
  'fcodeName': 'capital of a political entity',
  'adminName1': 'Île-de-France',
  'lat': '48.85341',
  'fcode': 'PPLC'},
 {'adminCode1': '93',
  'lng': '5.38107',
  'geonameId': 2995469,
  'toponymName': 'Marseille',
  'countryId': '3017382',
  'fcl': 'P',
  'population': 794811,
  'countryCode': 'FR',
  'name': 'Marseille',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': 'PAC'},
  'countryName': 'France',
  'fcodeName': 'seat of a first-order administrative division',
  'adminName1': "Provence-Alpes-Côte d'Azur",
  'lat': '43.29695',
  'fcode': 'PPLA'},
 {'adminCode1': '84',
  'lng': '4.84671',
  'geonameId': 2996944,
  'toponymName': 'Lyon',
  'countryId': '3017382',
  'fcl

In [20]:
resu = list(map(lambda x: {'geonameId': x['geonameId'], 'name': x[
            'name'], 'geometry': (x['lng']) + ' ' + x['lat']}, result))
resu

[{'geonameId': 2988507, 'name': 'Paris', 'geometry': '2.3488 48.85341'},
 {'geonameId': 2995469, 'name': 'Marseille', 'geometry': '5.38107 43.29695'},
 {'geonameId': 2996944, 'name': 'Lyon', 'geometry': '4.84671 45.74846'},
 {'geonameId': 2972315, 'name': 'Toulouse', 'geometry': '1.44367 43.60426'},
 {'geonameId': 2990440, 'name': 'Nice', 'geometry': '7.26608 43.70313'},
 {'geonameId': 2990969, 'name': 'Nantes', 'geometry': '-1.55336 47.21725'},
 {'geonameId': 2973783, 'name': 'Strasbourg', 'geometry': '7.74553 48.58392'},
 {'geonameId': 2992166, 'name': 'Montpellier', 'geometry': '3.87635 43.61093'},
 {'geonameId': 3031582, 'name': 'Bordeaux', 'geometry': '-0.5805 44.84044'},
 {'geonameId': 2998324, 'name': 'Lille', 'geometry': '3.05858 50.63297'}]

In [30]:
ssl_context = ssl.SSLContext()
ssl_context.verify_mode = ssl.CERT_NONE
conn = pg8000.connect(user = "p1711842", host = "bd-pedago.univ-lyon1.fr", port = 5432,
                              database = "p1711842", password = "Pu1WtY83xbUx", ssl_context=ssl_context)
localConnection = pg8000.connect(user = "postgres", host = "localhost", port = 5432,
                              database = "postgres", password = "password")
                

In [22]:
conn.run("CREATE TABLE if not exists public.villes (geonameid INT, geom Geometry, name character varying);")
conn.commit()

In [17]:

for city in resu:
    conn.run("INSERT INTO public.villes(geonameid, geom, name) Values({}, {}, '{}');".format(
        city['geonameId'], "ST_GeomFromText('POINT({})', 4326)".format(city['geometry']), city['name']))
conn.commit()

In [31]:

conn.run(
    "CREATE TABLE IF NOT EXISTS public.ville_distance_min AS SELECT vi.geonameid, vi.name, vi.geom, min(ST_DistanceSphere(ae.geom, vi.geom)) as dist_meters FROM  public.villes as vi, public.airports as ae GROUP BY vi.geonameid, vi.name, vi.geom;"
)
conn.commit()
conn.run(
    "CREATE TABLE IF NOT EXISTS public.aeroport_proche_ville AS SELECT vi.geom, vi.geonameid, ae.city, ae.airport_name, ae.icao, ST_DistanceSphere(ae.geom, vi.geom) FROM public.ville_distance_min as vi, public.airports as ae WHERE ST_DistanceSphere(ae.geom, vi.geom)=dist_meters; "
)
conn.commit()


In [9]:
# Methode 3 veronoi
conn.run("Select pv.airport_name, pv.id, pv.city, vi.\"name\", vi.geonameid from \"Polygones de Voronoï\" as pv INNER JOIN villes as vi ON ST_CONTAINS(pv.geom, vi.geom)")

(['Mediterranee', 1359, 'Montpellier', 'Montpellier', 2992166],
 ['Lasbordes', 1286, 'Toulouse', 'Toulouse', 2972315],
 ['Aix Les Milles', 1347, 'Aix-les-milles', 'Marseille', 2995469],
 ['Merignac', 1264, 'Bordeaux', 'Bordeaux', 3031582],
 ["Cote D\\\\'Azur", 1354, 'Nice', 'Nice', 2990440],
 ['Bron', 1346, 'Lyon', 'Lyon', 2996944],
 ['Nantes Atlantique', 1418, 'Nantes', 'Nantes', 2990969],
 ['Polygone', 7892, 'Strasbourg Neudorf', 'Strasbourg', 2973783],
 ['Orly', 1386, 'Paris', 'Paris', 2988507],
 ['Lesquin', 1399, 'Lille', 'Lille', 2998324])

In [33]:
conn.run("Select pv.airport_name, pv.id, pv.city, vi.name, vi.geom, vi.geonameid from public.voronoi as pv INNER JOIN public.villes as vi ON ST_CONTAINS(pv.geom, vi.geom)")

(['Orly',
  1386,
  'Paris',
  'Paris',
  '0101000020E610000027C286A757CA0240280AF4893C6D4840',
  2988507],
 ['Aix Les Milles',
  1347,
  'Aix-les-milles',
  'Marseille',
  '0101000020E610000067F2CD36378615400B46257502A64540',
  2995469],
 ['Bron',
  1346,
  'Lyon',
  'Lyon',
  '0101000020E6100000DEC83CF2076313409D2E8B89CDDF4640',
  2996944],
 ['Lasbordes',
  1286,
  'Toulouse',
  'Toulouse',
  '0101000020E61000000C76C3B64519F73FF623456458CD4540',
  2972315],
 ["Cote D\\\\'Azur",
  1354,
  'Nice',
  'Nice',
  '0101000020E61000008D7A884677101D40126BF12900DA4540',
  2990440],
 ['Nantes Atlantique',
  1418,
  'Nantes',
  'Nantes',
  '0101000020E61000000AA2EE0390DAF8BF2B8716D9CE9B4740',
  2990969],
 ['Polygone',
  7892,
  'Strasbourg Neudorf',
  'Strasbourg',
  '0101000020E61000005DBF60376CFB1E407BBDFBE3BD4A4840',
  2973783],
 ['Mediterranee',
  1359,
  'Montpellier',
  'Montpellier',
  '0101000020E6100000FBCBEEC9C3020F40991249F432CE4540',
  2992166],
 ['Merignac',
  1264,
  'Bordeaux',
  

In [ ]:


conn.close()
